# Введение

Здравствуйте, коллеги! Добро пожаловать на восьмой вебинар, посвященный работе с Git и инструментами для управления экспериментами в области машинного обучения. Сегодня мы рассмотрим:

Инструменты для проведения и оценки результатов экспериментов.
Data Version Control (DVC): надстройка над Git и примеры версионирования датасетов.
Логирование экспериментов с помощью DVC.
Автоматизация пайплайнов с DVC.
MLflow и жизненный цикл ML-модели.
Компоненты MLflow: Tracking, Projects, Models, Registry.

Инструменты для проведения и оценки результатов экспериментов

При разработке моделей машинного обучения важно обеспечивать воспроизводимость и управляемость экспериментов. Необходимо отслеживать изменения в коде, данных, параметрах модели и метриках. Для этого используются следующие инструменты:

Git: система контроля версий для кода.
DVC: расширение Git для управления данными и моделями.
MLflow: платформа для управления жизненным циклом ML-моделей.

Data Version Control (DVC)

## Что такое DVC?

DVC — это система управления версиями данных и моделей, которая интегрируется с Git. Она позволяет:

Отслеживать большие файлы данных и модели.
Управлять зависимостями и воспроизводить эксперименты.
Хранить метаданные в Git, а сами данные — в удаленном хранилище.

Установка и инициализация DVC

Установка DVC:
pip install dvc
Инициализация DVC в репозитории:

git init
dvc init
git commit -m "Initialize DVC"

Версионирование датасетов с DVC

1. Добавление датасета под управление DVC:
- dvc add data/raw/data.csv
- Команда создаст файл data/raw/data.csv.dvc и добавит data.csv в .gitignore.
2. Коммит изменений:

In [1]:
#git add data/raw/data.csv.dvc .gitignore
#git commit -m "Add raw data"

3. Настройка удаленного хранилища для данных:


In [2]:
#dvc remote add -d myremote s3://mybucket/dvcstore
#git commit .dvc/config -m "Configure remote storage"

4. Отправка данных в удаленное хранилище:
dvc push

Извлечение данных из репозитория

При клонировании репозитория можно загрузить данные:


In [3]:
#git clone https://github.com/user/repo.git
#dvc pull

## DVC Логирование экспериментов

DVC позволяет вести журнал экспериментов и сравнивать результаты.

Запуск и логирование экспериментов

Создание DVC-пайплайна:

In [4]:
#dvc run -n preprocess \
#    -d src/preprocess.py -d data/raw/data.csv \
#    -o data/processed/data_processed.csv \
#    python src/preprocess.py

Обучение модели:

In [5]:
#dvc run -n train \
#    -d src/train.py -d data/processed/data_processed.csv \
#    -o models/model.pkl \
#    -M metrics/metrics.json \
#    python src/train.py

Здесь -M указывает на файл с метриками, который будет отслеживаться DVC.

Просмотр и сравнение метрик

1. Просмотр метрик последнего эксперимента:
dvc metrics show
2. Сравнение метрик между разными коммитами:
dvc metrics diff HEAD~1

## DVC Автоматизация пайплайнов

Создание полного пайплайна

DVC позволяет описать весь процесс от обработки данных до обучения модели.


In [6]:
#dvc run -n evaluate \
#    -d src/evaluate.py -d models/model.pkl -d data/test \
#    -M metrics/eval.json \
#    python src/evaluate.py

Визуализация пайплайна

dvc dag

Эта команда отобразит DAG (Directed Acyclic Graph) ваших этапов.

Запуск пайплайна при изменениях

Если изменились исходные данные или код, можно воспроизвести весь пайплайн:

dvc repro

## MLflow: Жизненный цикл ML-модели

MLflow предоставляет инструменты для управления экспериментами, воспроизводимости и деплоя моделей.

Компоненты MLflow:

MLflow Tracking: логирование параметров, метрик и артефактов экспериментов.
MLflow Projects: упаковка кода в формат, удобный для воспроизводимости.
MLflow Models: стандарт для сохранения и развертывания моделей.
MLflow Registry: хранение и управление версиями моделей.

MLflow Tracking

Установка MLflow

pip install mlflow

Логирование экспериментов


In [8]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor

# Начало нового эксперимента
with mlflow.start_run():
    # Параметры модели
    params = {"n_estimators": 100, "max_depth": 5}
    mlflow.log_params(params)
    
    # Обучение модели
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    
    # Оценка метрик
    rmse = evaluate(model, X_test, y_test)
    mlflow.log_metric("rmse", rmse)
    
    # Сохранение модели
    mlflow.sklearn.log_model(model, "random_forest_model")

NameError: name 'X_train' is not defined

## Запуск MLflow UI

mlflow ui

После запуска интерфейс будет доступен по адресу http://localhost:5000.

MLflow Projects

Структура проекта

1. Создание файла MLproject:

name: RandomForestProject

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      n_estimators: {type: int, default: 100}
      max_depth: {type: int, default: 5}
    command: "python train.py --n_estimators {n_estimators} --max_depth {max_depth}"

2. Создание файла conda.yaml:

name: mlflow_env
channels:
  - defaults
dependencies:
  - python=3.8
  - scikit-learn
  - pip
  - pip:
    - mlflow

Запуск проекта с разными параметрами

mlflow run . -P n_estimators=200 -P max_depth=7

MLflow Models

Сохранение модели в формате MLflow

mlflow.sklearn.log_model(model, artifact_path="models")

Развертывание модели

MLflow поддерживает развертывание моделей с использованием различных фреймворков.

Пример развертывания модели как REST API:

mlflow models serve -m runs:/<run_id>/models --port 1234

Теперь можно делать запросы к модели через HTTP.

MLflow Registry

Регистрация модели

In [ ]:
result = mlflow.register_model(
    model_uri="runs:/<run_id>/models",
    name="RandomForestModel"
)

Управление версиями и стадиями модели

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.transition_model_version_stage(
    name="RandomForestModel",
    version=1,
    stage="Production"
)